## 서울생활이동 데이터
- https://data.seoul.go.kr/dataVisual/seoul/seoulLivingMigration.do

#### 해당 데이터는 '서울 열린데이터 광장' 에서 제공하는 .csv 파일 형태로 수집함.
- 수집한 데이터
    - 데이터의 크기가 커서 업로드된 데이터 파일 중 가장 최신의 3개월간의 데이터를 수집
        - 2023년04월 ~ 2023년05월 3개월에 대한 데이터<br><br>
        
- 수집한 데이터를 가공
    - 필요한 데이터를 추출, 각 column들 중 필요한 것들만 추출
    - 여러 데이터 파일을 하나의 데이터로 만들기<br><br>
- 데이터를 분석하기 알맞게 전처리
    - field를 새로 만들거나 가공함(분석에 용이하도록)
    - 결측값과 같이 분석하는 것에 지장을 주는 부분들 체크

In [1]:
import pandas as pd
import numpy as np 

In [2]:
# 경고 메시지 무시 설정
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 6월 데이터
data_0608 =  pd.read_csv('./data/생활이동_행정동_2023.06_08시.csv', encoding='cp949')
data_0609 =  pd.read_csv('./data/생활이동_행정동_2023.06_09시.csv', encoding='cp949')
data_0610 =  pd.read_csv('./data/생활이동_행정동_2023.06_10시.csv', encoding='cp949')

# 5월 데이터
data_0508 =  pd.read_csv('./data/생활이동_행정동_2023.05_08시.csv', encoding='cp949')
data_0509 =  pd.read_csv('./data/생활이동_행정동_2023.05_09시.csv', encoding='cp949')
data_0510 =  pd.read_csv('./data/생활이동_행정동_2023.05_10시.csv', encoding='cp949')

# 4월 데이터
data_0408 =  pd.read_csv('./data/생활이동_행정동_2023.04_08시.csv', encoding='cp949')
data_0409 =  pd.read_csv('./data/생활이동_행정동_2023.04_09시.csv', encoding='cp949')
data_0410 =  pd.read_csv('./data/생활이동_행정동_2023.04_10시.csv', encoding='cp949')

In [4]:
# 데이터 확인
data_0608

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202306,일,8,1101053,1101053,F,10,HH,2,5.93
1,202306,일,8,1101053,1101053,F,25,EH,5,*
2,202306,일,8,1101053,1101053,F,30,HE,7,*
3,202306,일,8,1101053,1101053,F,30,HH,10,*
4,202306,일,8,1101053,1101053,F,35,EW,12,3.02
...,...,...,...,...,...,...,...,...,...,...
6319097,202306,토,8,1117061,1117056,F,20,HW,14,6.60
6319098,202306,토,8,1117061,1117056,F,30,EH,27,*
6319099,202306,토,8,1117061,1117056,F,30,HE,25,*
6319100,202306,토,8,1117061,1117056,F,30,HW,21,7.20


## 데이터를 그냥 합치면 크기가 너무 커져서 몇가지의 기준으로 데이터 필터링 후 합치기

- 데이터는 <b>출근시간</b>을 추출.<br><br>

#### 'HW' [야간상주지(집) => 주간상주지(근무지)]  가 출근이므로 해당 조건으로 필터링을 진행한다.
- 야간 근무자, 3교대 근무자 등 시간적으로 유동적 특성이 있는 직군들을 고려하기엔 출근 시간에 대해 일괄적인 기준을 적용하기가 어렵고
- 현재 분석하고자 하는 방향이 <b>평일 오전 8~10시 출근으로 혼잡한 시간대를 타겟</b>으로 삼고 있기 때문에, 9-18시 주간근무자들을 기준으로 출근시간을 정의. <b>주말을 제외한 경우는 출근이 아닌 약속이나 여행에 의한 이동까지도 포함 되었을 확률</b>이 높기 때문이다.
<br><br>

#### 이동유형은 W(주간상주지), H(야간상주지), E(기타지역)의 세가지 지역간의 이동을 의미한다.
- 주간상주지 : 새벽 5시 이후에 체류를 시작하여 누적시간이 가장 많은 장소 (근무지, 학교 등으로 추정)
- 야간상주지 : 새벽 5시를 포함하며 체류한 누적시간이 가장 많은 장소 (거주지, 집으로 추정)
- 기타지역 :  주간, 야간 상주지로 설정되지 않은 나머지 시간에 체류한 지역

In [5]:
condition = (data_0608['이동유형'] == 'HW')
# condition
df_0608 = data_0608.loc[condition,:]

In [6]:
# 정상적으로 필터링 진행 확인
df_0608.head(3)

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
8,202306,일,8,1101053,1101053,F,40,HW,5,13.55
16,202306,일,8,1101053,1101053,F,50,HW,9,3.10
21,202306,일,8,1101053,1101053,F,55,HW,9,19.39


### 같은 방식으로 나머지 data에 적용.

In [7]:
# 빈 df생성
df_gtw = pd.DataFrame()
df_gtw

""


In [8]:
# 출근 이동유형의 데이터만 필터링하는 함수 정의
def go_to_work(df):
    temp = df.loc[(df['이동유형'] == 'HW')&(df['요일'] != '일')&(df['요일'] != '토'),:]
    global df_gtw 
    df_gtw = pd.concat([df_gtw, temp], axis=0, ignore_index=True)
    return df_gtw    

In [9]:
# 각 data에 함수 실행
go_to_work(data_0608)
go_to_work(data_0609)
go_to_work(data_0610)
go_to_work(data_0508)
go_to_work(data_0509)
go_to_work(data_0510)
go_to_work(data_0408)
go_to_work(data_0409)
go_to_work(data_0410)

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202306,월,8,1101053,1101053,F,0,HW,2,27.83
1,202306,월,8,1101053,1101053,F,10,HW,7,5.94
2,202306,월,8,1101053,1101053,F,25,HW,35,16.28
3,202306,월,8,1101053,1101053,F,30,HW,14,23.62
4,202306,월,8,1101053,1101053,F,35,HW,7,18.04
...,...,...,...,...,...,...,...,...,...,...
19235681,202304,금,10,38000,1101064,M,35,HW,289,*
19235682,202304,금,10,38000,1122053,M,55,HW,294,6.59
19235683,202304,금,10,39000,1102055,F,35,HW,231,3.03
19235684,202304,금,10,1114063,3110354,M,30,HW,71,*


In [10]:
df_gtw.head()
df_gtw.tail()
df_gtw['이동유형'].unique()

array(['HW'], dtype=object)

In [11]:
# 이동유형 column은 값이 하나만 있으므로 필요없음
df_gtw = df_gtw[['대상연월', '요일', '출발 행정동 코드', '도착 행정동 코드', '성별', '나이', '평균 이동 시간(분)', \
                 '이동인구(합)']]

#### 이동인구(합) 데이터에 '*' 로 기입되어 있는 데이터는  3명 미만의 인구로, 데이터 전체의 크기가 부족하지 않으므로 2로 대체해서 사용. 그리고 실수로 입력된 인구수는 round()로 반올림 처리

In [12]:
# 이동인구(합) column 에 '*'이 얼마나 있을까?
# 2865420 개의 '*' 데이터, 전체 데이터가 19235686 개 이므로 전체에 비해 비율적으로 많은 수가 아니고,
# 데이터가 1명 또는 2명이기때문에 2명으로 설정하고 진행해도 무방하다고 판단.
df_gtw['이동인구(합)'].loc[df_gtw['이동인구(합)'] == '*'] = 2

In [13]:
# 이동인구(합) column 데이터를 round() 로 반올림처리 후 int로 형변환
df_gtw['이동인구(합)'] = df_gtw['이동인구(합)'].astype(float).round().astype(int)

In [14]:
# 이동인구(합) column의 데이터타입이 int인 것을 확인
df_gtw['이동인구(합)'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 19235686 entries, 0 to 19235685
Series name: 이동인구(합)
Non-Null Count     Dtype
--------------     -----
19235686 non-null  int32
dtypes: int32(1)
memory usage: 73.4 MB


### 출근을 타겟으로 두고 있기 때문에, '나이' column의 데이터를 20세 이상으로 필터링

In [15]:
# loc 조건에 맞는 행들만 다시 df_gtw에 저장
df_gtw = df_gtw.loc[df_gtw['나이']>=20,:]
df_gtw.reset_index(drop=True, inplace=True)
df_gtw.head()

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합)
0,202306,월,1101053,1101053,F,25,35,16
1,202306,월,1101053,1101053,F,30,14,24
2,202306,월,1101053,1101053,F,35,7,18
3,202306,월,1101053,1101053,F,40,19,27
4,202306,월,1101053,1101053,F,45,15,16


In [16]:
df_gtw.columns

Index(['대상연월', '요일', '출발 행정동 코드', '도착 행정동 코드', '성별', '나이', '평균 이동 시간(분)',
       '이동인구(합)'],
      dtype='object')

In [17]:
df_gtw['나이'].unique()

array([25, 30, 35, 40, 45, 50, 55, 60, 65, 20, 80, 75, 70], dtype=int64)

### 출근 가능 지역 필터링

#### 출발지역과 도착지역으로 데이터가 수집되어 있음
- 생활 이동 데이터이기 때문에 출근시간동안 일어난 이동이라고 해도 목적이 출근이 아닌 경우가 있기 때문에,
- 일반적인 출근거리를 넘어서는 지역의 움직임은 제거 후 사용
<br><br>

#### 출근 가능 지역이 아닌 것으로 판단하여 제거하는 지역
- 32000	강원도<br>
- 33000	충청북도<br>
- 34000	충청남도<br>
- 35000	전라북도<br>
- 36000	전라남도<br>
- 37000	경상북도<br>
- 38000	경상남도<br>
- 39000	제주특별자치도<br>
- 24000	광주광역시<br>
- 25000	대전광역시<br>
- 26000	울산광역시<br>
- 21000	부산광역시<br>
- 22000	대구광역시<br>



In [18]:
# 제거할 지역 코드 리스트
reg_list = [32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 24000, 25000, \
           26000, 21000, 22000]

In [19]:
# 출발 행정동 코드가 리스트에 포함되지 않은 행들만 필터링
df_gtw = df_gtw.loc[~df_gtw['출발 행정동 코드'].isin(reg_list),:]

In [20]:
# 도착 행정동 코드가 리스트에 포함되지 않은 행들만 필터링
df_gtw = df_gtw.loc[~df_gtw['도착 행정동 코드'].isin(reg_list),:]

In [21]:
# 필터링 되어 리스트와 같은 행정동 코드가 남아있지 않음을 알 수 있음
df_gtw.loc[df_gtw['출발 행정동 코드'].isin(reg_list),:]

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합)


In [22]:
df_gtw

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합)
0,202306,월,1101053,1101053,F,25,35,16
1,202306,월,1101053,1101053,F,30,14,24
2,202306,월,1101053,1101053,F,35,7,18
3,202306,월,1101053,1101053,F,40,19,27
4,202306,월,1101053,1101053,F,45,15,16
...,...,...,...,...,...,...,...,...
18459677,202304,금,3138033,1121078,M,45,117,3
18459678,202304,금,3138033,1122053,M,55,101,3
18459679,202304,금,3138033,1122053,M,60,80,4
18460031,202304,금,1114063,3110354,M,30,71,2


#### 인천의 옹진군은 너무 멀어 제외 , 세종시도 제외

In [23]:
reg_list2 = [2332031, 2332032,2332033,2332034,2332035,2332036,2332037,29000]

In [24]:
# 출발 행정동 코드가 리스트에 포함되지 않은 행들만 필터링
df_gtw = df_gtw.loc[~df_gtw['출발 행정동 코드'].isin(reg_list2),:]
# 도착 행정동 코드가 리스트에 포함되지 않은 행들만 필터링
df_gtw = df_gtw.loc[~df_gtw['도착 행정동 코드'].isin(reg_list2),:]

In [25]:
# 필터링 되어 리스트와 같은 행정동 코드가 남아있지 않음을 알 수 있음
df_gtw.loc[df_gtw['출발 행정동 코드'].isin(reg_list2),:]

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합)


In [26]:
df_gtw

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합)
0,202306,월,1101053,1101053,F,25,35,16
1,202306,월,1101053,1101053,F,30,14,24
2,202306,월,1101053,1101053,F,35,7,18
3,202306,월,1101053,1101053,F,40,19,27
4,202306,월,1101053,1101053,F,45,15,16
...,...,...,...,...,...,...,...,...
18459677,202304,금,3138033,1121078,M,45,117,3
18459678,202304,금,3138033,1122053,M,55,101,3
18459679,202304,금,3138033,1122053,M,60,80,4
18460031,202304,금,1114063,3110354,M,30,71,2


### 출발/도착 행정동 코드에 대해 어떤 지역인지 알 수있는 지역명 column으로 변경

#### 1. '출발 지역명'
#### 2. '도착 지역명'

In [27]:
df_reg = pd.read_excel('./data/서울생활이동데이터_행정동코드_20210907.xlsx', usecols=['읍면동', 'full_name'])

In [28]:
df_reg.head()
df_reg.rename(columns = {'읍면동' : '출발 행정동 코드'}, inplace = True)
df_reg.head()

,출발 행정동 코드,full_name
0,1101053,종로구
1,1101054,종로구
2,1101055,종로구
3,1101056,종로구
4,1101057,종로구


In [29]:
df_reg.dropna(inplace=True)
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 0 to 1143
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   출발 행정동 코드  1144 non-null   int64 
 1   full_name  1144 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.8+ KB


### 행정동 코드 정보 데이터를 가져와 출발/도착 정보를 지역명으로 대체

In [30]:
df_start = pd.merge(df_gtw, df_reg, on='출발 행정동 코드')
df_start.head()

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합),full_name
0,202306,월,1101053,1101053,F,25,35,16,종로구
1,202306,월,1101053,1101053,F,30,14,24,종로구
2,202306,월,1101053,1101053,F,35,7,18,종로구
3,202306,월,1101053,1101053,F,40,19,27,종로구
4,202306,월,1101053,1101053,F,45,15,16,종로구


In [31]:
df_start.rename(columns = {'full_name' : '출발 지역명'}, inplace = True)
df_start

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합),출발 지역명
0,202306,월,1101053,1101053,F,25,35,16,종로구
1,202306,월,1101053,1101053,F,30,14,24,종로구
2,202306,월,1101053,1101053,F,35,7,18,종로구
3,202306,월,1101053,1101053,F,40,19,27,종로구
4,202306,월,1101053,1101053,F,45,15,16,종로구
...,...,...,...,...,...,...,...,...,...
18353827,202304,월,3135038,1111060,F,65,134,5,연천군
18353828,202305,수,2331041,1115054,M,65,116,4,강화군
18353829,202304,월,2331041,1101056,F,65,143,5,강화군
18353830,202304,월,2331041,1122051,F,60,194,8,강화군


In [32]:
df_reg.rename(columns = {'출발 행정동 코드' : '도착 행정동 코드'}, inplace = True)
df = pd.merge(df_start, df_reg, on='도착 행정동 코드')
df

,대상연월,요일,출발 행정동 코드,도착 행정동 코드,성별,나이,평균 이동 시간(분),이동인구(합),출발 지역명,full_name
0,202306,월,1101053,1101053,F,25,35,16,종로구,종로구
1,202306,월,1101053,1101053,F,30,14,24,종로구,종로구
2,202306,월,1101053,1101053,F,35,7,18,종로구,종로구
3,202306,월,1101053,1101053,F,40,19,27,종로구,종로구
4,202306,월,1101053,1101053,F,45,15,16,종로구,종로구
...,...,...,...,...,...,...,...,...,...,...
18353827,202304,화,1122054,2305061,F,60,89,8,서초구,남동구
18353828,202304,수,1122054,2305061,F,60,110,8,서초구,남동구
18353829,202304,목,1122054,2305061,F,60,77,8,서초구,남동구
18353830,202304,금,1122054,2305061,F,60,76,8,서초구,남동구


In [33]:
df.rename(columns = {'full_name' : '도착 지역명'}, inplace = True)

In [34]:
df = df[['대상연월', '요일', '성별', '나이', '평균 이동 시간(분)', '이동인구(합)', '출발 지역명', '도착 지역명']]

In [35]:
df

,대상연월,요일,성별,나이,평균 이동 시간(분),이동인구(합),출발 지역명,도착 지역명
0,202306,월,F,25,35,16,종로구,종로구
1,202306,월,F,30,14,24,종로구,종로구
2,202306,월,F,35,7,18,종로구,종로구
3,202306,월,F,40,19,27,종로구,종로구
4,202306,월,F,45,15,16,종로구,종로구
...,...,...,...,...,...,...,...,...
18353827,202304,화,F,60,89,8,서초구,남동구
18353828,202304,수,F,60,110,8,서초구,남동구
18353829,202304,목,F,60,77,8,서초구,남동구
18353830,202304,금,F,60,76,8,서초구,남동구


In [36]:
# 서울지역 시군구 리스트
sgg_list = ['종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구','도봉구','노원구','은평구','서대문구'\
            ,'마포구','양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구','강남구','송파구','강동구']
sgg_list

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

### 서울로 출근하는 인원이 타겟이기 때문에 '도착 지역명'이 서울내 지역구인 데이터만 추출

In [37]:
df = df.loc[df['도착 지역명'].isin(sgg_list),:]

In [38]:
df.head()

,대상연월,요일,성별,나이,평균 이동 시간(분),이동인구(합),출발 지역명,도착 지역명
0,202306,월,F,25,35,16,종로구,종로구
1,202306,월,F,30,14,24,종로구,종로구
2,202306,월,F,35,7,18,종로구,종로구
3,202306,월,F,40,19,27,종로구,종로구
4,202306,월,F,45,15,16,종로구,종로구


### 데이터 중, 거리상으로나 거리가 멀다고 가정해도 너무 많은 소요시간을 보이는 데이터들이 있다.
- 이동 자체가 업무인 직종들(운송업 등)
- 이동 자체가 취미나 운동인 경우들(자전거, 바이크 라이딩 등)
<br><br>

#### 위와 같은 예라고 판단하여 출근이라고 할만한 시간 기준을 넉넉하게 3시간으로 보고 3시간 이하의 평균시간을 가진 데이터만추출

In [39]:
df = df.loc[df['평균 이동 시간(분)'] < 180,]

In [40]:
df.reset_index(drop=True, inplace=True)

In [41]:
df

,대상연월,요일,성별,나이,평균 이동 시간(분),이동인구(합),출발 지역명,도착 지역명
0,202306,월,F,25,35,16,종로구,종로구
1,202306,월,F,30,14,24,종로구,종로구
2,202306,월,F,35,7,18,종로구,종로구
3,202306,월,F,40,19,27,종로구,종로구
4,202306,월,F,45,15,16,종로구,종로구
...,...,...,...,...,...,...,...,...
16016619,202306,금,M,60,152,3,강남구,중구
16016620,202305,월,M,60,95,3,강남구,중구
16016621,202305,월,M,60,149,3,강남구,중구
16016622,202304,수,M,60,107,3,강남구,중구


### 전처리 후 데이터의  컬럼에 대한 정보 정리
<hr>
<br>

- <b>대상연월</b> : 취합된 년과 월 정보 데이터 ex) 202306 
- <b>요일</b> : 월~금요일 5개의 명목형 데이터 ex) '월'
- <b>성별</b> : 이동한 사람의 성별 ex) '남'
- <b>나이</b> : 이동한 사람의 나이(5세 단위)  ex) 35 : 35~39세
- <b>평균 이동 시간(분)</b> : 이동에 소요된 시간 ex) 75
- <b>이동인구(합)</b> : 상위의 column들의 특성을 공통으로 갖는 이동인구의 합 데이터 ex) 5
- <b>출발 지역명</b> : 출발한 체류지역명	(시군구 단위)
- <b>도착 지역명</b> : 도착한 체류지역명 (시군구 단위)

In [42]:
# df.to_csv('./data/서울생활이동_전처리데이터.csv')

## 간단한 전처리 작업을 하며 고려한 사항 정리

- 데이터는 출근 시간을 기준으로 추출
- 월 ~ 금요일 데이터만 추출
- 나이를 15세 이상으로 추출
- 일반적으로 출근이 가능하지 않은 지역을 제외하고 추출 (출근외 다른목적 확률 높음)
- 도착하는 지역명(출근 목적지)이 서울인 데이터만 추출